## Scrape beer information off BeerAdvocate using BeautifulSoup4

In this walkthrough, we will scrape beer reviews and data fields off [beeradvocate.com](https://www.beeradvocate.com/)

Beer Advocate is a website which collects reviews and data about beers from all over the world. They rate these beers based on user feedback and provides useful information about the type of beer, the brewery which produces it and the region where it is produced.

We are going to be utilizing [BeatifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) which makes parsing HTML documents very easy and also provides the `Request` class which helps us get the web page using the url.

### _At the time of scraping, the website provided a list of about ~6.5k different beers_

### Main concepts used: web scraping, multi-threading

### Output of the exercise: beer_advocate.csv

We are going to create a dataset which has the following pieces of information:
- **beer_name**: _name of the beer_
- **brewery**: _name of the brewery_
- **ba_score**: _beer advocate score: scale of 1 to 5_
- **ranking**: _overall ranking of the beer_
- **reviews**: _number of reviews by users_
- **ratings**: _number of ratings by users_
- **pdev**: _percent deviation in ratings_
- **wants**: _no of users who want this beer_
- **gots**: _no of users who have had/got this beer_
- **trade**: _no of users interested in a trade_
- **location**: _location of the beer or region_
- **style**: _beer style_
- **alcohol**: _alcohol content_
- **availability**: _is it available?_
- **note_desc**: _detailed description of the beer_


In [1]:
from bs4 import BeautifulSoup
from collections import namedtuple
import requests
import re
import pandas as pd
import threading

In [2]:
# We define a namedtuple for the beer instead of a class
Beer = namedtuple('Beer', ['beer_name', 'brewery', 'ba_score', 'ranking', 'reviews', 'ratings', 'pdev',
                           'wants', 'gots', 'trade', 'location', 'style', 'alcohol', 'availability',
                           'note_desc'])

In [3]:
def scrape_beer_page(page_url):
    """
    THe function scrapes the beer information from a beer page. 
    
    :param page_url: beer page to scrape
    :return: beer object which contains the scraped information
    """

    page_response = requests.get(page_url, timeout=5)

    full_soup = BeautifulSoup(page_response.content, 'lxml')

    title = full_soup.find('div', class_='titleBar')

    beer_name = list(title.h1.strings)[0].strip()
    brewery = title.span.string.replace('|', '').strip()

    ba_content = full_soup.find(id='ba-content')
    ba_score = ba_content.find(id='score_box').find('span', class_='ba-ravg').string

    item_stats_content = ba_content.find(id='item_stats')

    stats = ['Ranking', 'Reviews', 'Ratings', 'pDev', 'Wants', 'Gots', 'Trade']

    for elem in item_stats_content.find_all('dt'):
        stat_name = elem.next_element.string.replace(':', '')
        stat_value = elem.find_next('dd').string.strip()
        if stat_name == 'Ranking':
            ranking = stat_value
        elif stat_name == 'Reviews':
            reviews = stat_value
        elif stat_name == 'Ratings':
            ratings = stat_value
        elif stat_name == 'pDev':
            pdev = stat_value
        elif stat_name == 'Wants':
            wants = stat_value
        elif stat_name == 'Gots':
            gots = stat_value
        elif stat_name == 'Trade':
            trade = stat_value

    # Info-box
    info_box = full_soup.find(id='info_box')

    location_marker = info_box.find(string=re.compile("Brewed by"))
    location = location_marker.find_next('a').find_next('a').string

    style_marker = info_box.find('b', string='Style:')
    style = style_marker.find_next('a').string.strip()

    alc_marker = info_box.find('b', string=re.compile("Alcohol by"))
    alcohol = alc_marker.next_sibling.strip()

    avail_marker = info_box.find('b', string=re.compile("Availability"))
    availability = avail_marker.next_sibling.strip()

    note_marker = info_box.find('b', string=re.compile("Notes / Commercial"))
    note_desc = note_marker.find_next('br').next_element

#     print(f'Beer Name: {beer_name}')
#     print(f'Brewery: {brewery}')
#     print(f'BA Score: {ba_score}')

#     print(f'Ranking: {ranking}')
#     print(f'Reviews: {reviews}')
#     print(f'Ratings: {ratings}')
#     print(f'pDev: {pdev}')
#     print(f'Wants {wants}')
#     print(f'Gots: {gots}')
#     print(f'Trade: {trade}')

#     print(f'Location: {location}')
#     print(f'Style: {style}')
#     print(f'Alcohol Percentage: {alcohol}')
#     print(f'Availability: {availability}')
#     print(f'Notes: {note_desc}')

    return Beer(beer_name, brewery, ba_score, ranking, reviews, ratings, pdev,
                wants, gots, trade, location, style, alcohol, availability,
                note_desc)

In [4]:
page_url = 'https://www.beeradvocate.com/beer/profile/16043/112329/'
beer = scrape_beer_page(page_url)
print(beer)

Beer(beer_name='Chrysopolis', brewery='Birrificio Del Ducato', ba_score='4.1', ranking='-', reviews='19', ratings='106', pdev='10.24%', wants='3', gots='17', trade='0', location='Italy', style='Belgian Lambic', alcohol='5.00%', availability='Limited (brewed once)', note_desc='\nNo notes at this time.')


In [5]:
def scrape_beer_urls(page_url):
    """
    This function gets the urls from a list of beer urls and puts them in a list.
    """
    page_response = requests.get(page_url, timeout=5)

    full_soup = BeautifulSoup(page_response.content, 'lxml')

    beer_table = full_soup.find('div', id='ba-content').table

    def is_a_and_parent_is_td(tag):
        return tag.parent.name == 'td' and tag.name == 'a'

    all_beers = beer_table.find_all(is_a_and_parent_is_td)
    beer_urls = []

    for beer_tag in all_beers:
        beer_urls.append('https://www.beeradvocate.com' + beer_tag.get('href'))
        print('https://www.beeradvocate.com' + beer_tag.get('href'))

    return beer_urls

In [6]:
url = 'https://www.beeradvocate.com/lists/top/'
beer_urls = scrape_beer_urls(url)

https://www.beeradvocate.com/beer/profile/23222/78820/
https://www.beeradvocate.com/beer/profile/26/42349/
https://www.beeradvocate.com/beer/profile/25888/87246/
https://www.beeradvocate.com/beer/profile/28743/87846/
https://www.beeradvocate.com/beer/profile/17981/110635/
https://www.beeradvocate.com/beer/profile/46317/16814/
https://www.beeradvocate.com/beer/profile/28743/146770/
https://www.beeradvocate.com/beer/profile/28743/237238/
https://www.beeradvocate.com/beer/profile/2216/188570/
https://www.beeradvocate.com/beer/profile/33824/172669/
https://www.beeradvocate.com/beer/profile/23222/162502/
https://www.beeradvocate.com/beer/profile/863/21690/
https://www.beeradvocate.com/beer/profile/23222/76421/
https://www.beeradvocate.com/beer/profile/32409/271487/
https://www.beeradvocate.com/beer/profile/28743/122114/
https://www.beeradvocate.com/beer/profile/1146/57747/
https://www.beeradvocate.com/beer/profile/28743/207976/
https://www.beeradvocate.com/beer/profile/36710/263446/
https:/

In [7]:
def scrape_beer_lists(page_url):
    """
    This scrapes the urls of the beer lists on the right pane of the page
    """
    page_response = requests.get(page_url, timeout=5)

    full_soup = BeautifulSoup(page_response.content, 'lxml')

    lists_section = full_soup.find('div', class_='secondaryContent')

    def is_a_and_href_list(tag):
        return tag.name == 'a' and tag.get('href').startswith('/lists/')

    all_beers = lists_section.find_all(is_a_and_href_list)
    beer_lists = []

    for beer_tag in all_beers:
        beer_lists.append('https://www.beeradvocate.com' + beer_tag.get('href'))
        # print('https://www.beeradvocate.com' + beer_tag.get('href'))

    return beer_lists

In [8]:
link = 'https://www.beeradvocate.com/lists/top/'
scrape_beer_lists(link)

['https://www.beeradvocate.com/lists/top/',
 'https://www.beeradvocate.com/lists/new/',
 'https://www.beeradvocate.com/lists/fame/',
 'https://www.beeradvocate.com/lists/popular/',
 'https://www.beeradvocate.com/lists/bottom/',
 'https://www.beeradvocate.com/lists/us/',
 'https://www.beeradvocate.com/lists/great-lakes/',
 'https://www.beeradvocate.com/lists/mid-atlantic/',
 'https://www.beeradvocate.com/lists/midwest/',
 'https://www.beeradvocate.com/lists/mountain/',
 'https://www.beeradvocate.com/lists/new-england/',
 'https://www.beeradvocate.com/lists/northwest/',
 'https://www.beeradvocate.com/lists/pacific/',
 'https://www.beeradvocate.com/lists/south/',
 'https://www.beeradvocate.com/lists/south-atlantic/',
 'https://www.beeradvocate.com/lists/southwest/',
 'https://www.beeradvocate.com/lists/state/al/',
 'https://www.beeradvocate.com/lists/state/ak/',
 'https://www.beeradvocate.com/lists/state/az/',
 'https://www.beeradvocate.com/lists/state/ar/',
 'https://www.beeradvocate.com

In [9]:
page_url = r'https://www.beeradvocate.com/lists/top/'

beer_list_urls = scrape_beer_lists(page_url)
unique_beers = set()
non_unique_beers = list()

for num, beer_list_url in enumerate(beer_list_urls):
    beers = scrape_beer_urls(beer_list_url)
    unique_beers = unique_beers.union(beers)
    non_unique_beers.append(beers)

with open('beer_urls.csv', 'w+') as f:
        f.write(str(unique_beers))

with open('non_unique_beer_urls.csv', 'w+') as f:
        f.write(str(non_unique_beers))

https://www.beeradvocate.com/beer/profile/23222/78820/
https://www.beeradvocate.com/beer/profile/26/42349/
https://www.beeradvocate.com/beer/profile/25888/87246/
https://www.beeradvocate.com/beer/profile/28743/87846/
https://www.beeradvocate.com/beer/profile/17981/110635/
https://www.beeradvocate.com/beer/profile/46317/16814/
https://www.beeradvocate.com/beer/profile/28743/146770/
https://www.beeradvocate.com/beer/profile/28743/237238/
https://www.beeradvocate.com/beer/profile/2216/188570/
https://www.beeradvocate.com/beer/profile/33824/172669/
https://www.beeradvocate.com/beer/profile/23222/162502/
https://www.beeradvocate.com/beer/profile/863/21690/
https://www.beeradvocate.com/beer/profile/23222/76421/
https://www.beeradvocate.com/beer/profile/32409/271487/
https://www.beeradvocate.com/beer/profile/28743/122114/
https://www.beeradvocate.com/beer/profile/1146/57747/
https://www.beeradvocate.com/beer/profile/28743/207976/
https://www.beeradvocate.com/beer/profile/36710/263446/
https:/

https://www.beeradvocate.com/beer/profile/26/345294/
https://www.beeradvocate.com/beer/profile/28743/322847/
https://www.beeradvocate.com/beer/profile/24822/316306/
https://www.beeradvocate.com/beer/profile/30356/314813/
https://www.beeradvocate.com/beer/profile/28743/350694/
https://www.beeradvocate.com/beer/profile/32409/332100/
https://www.beeradvocate.com/beer/profile/30654/360290/
https://www.beeradvocate.com/beer/profile/42273/334379/
https://www.beeradvocate.com/beer/profile/28743/371165/
https://www.beeradvocate.com/beer/profile/32319/312794/
https://www.beeradvocate.com/beer/profile/26676/320826/
https://www.beeradvocate.com/beer/profile/36710/327893/
https://www.beeradvocate.com/beer/profile/30654/325511/
https://www.beeradvocate.com/beer/profile/34258/349395/
https://www.beeradvocate.com/beer/profile/30654/367439/
https://www.beeradvocate.com/beer/profile/20680/315318/
https://www.beeradvocate.com/beer/profile/32409/316514/
https://www.beeradvocate.com/beer/profile/49747/320

https://www.beeradvocate.com/beer/profile/388/5281/
https://www.beeradvocate.com/beer/profile/863/7971/
https://www.beeradvocate.com/beer/profile/388/3659/
https://www.beeradvocate.com/beer/profile/1146/10672/
https://www.beeradvocate.com/beer/profile/313/1545/
https://www.beeradvocate.com/beer/profile/388/8954/
https://www.beeradvocate.com/beer/profile/1199/11757/
https://www.beeradvocate.com/beer/profile/2216/8773/
https://www.beeradvocate.com/beer/profile/207/645/
https://www.beeradvocate.com/beer/profile/313/857/
https://www.beeradvocate.com/beer/profile/259/1708/
https://www.beeradvocate.com/beer/profile/388/1703/
https://www.beeradvocate.com/beer/profile/863/9474/
https://www.beeradvocate.com/beer/profile/388/2558/
https://www.beeradvocate.com/beer/profile/216/217800/
https://www.beeradvocate.com/beer/profile/2097/8388/
https://www.beeradvocate.com/beer/profile/252/731/
https://www.beeradvocate.com/beer/profile/1141/10325/
https://www.beeradvocate.com/beer/profile/26/1558/
https:

https://www.beeradvocate.com/beer/profile/1199/11757/
https://www.beeradvocate.com/beer/profile/64/2093/
https://www.beeradvocate.com/beer/profile/863/7971/
https://www.beeradvocate.com/beer/profile/287/1093/
https://www.beeradvocate.com/beer/profile/46317/16814/
https://www.beeradvocate.com/beer/profile/199/29619/
https://www.beeradvocate.com/beer/profile/287/17112/
https://www.beeradvocate.com/beer/profile/1199/19960/
https://www.beeradvocate.com/beer/profile/1146/10672/
https://www.beeradvocate.com/beer/profile/112/412/
https://www.beeradvocate.com/beer/profile/64/6108/
https://www.beeradvocate.com/beer/profile/140/276/
https://www.beeradvocate.com/beer/profile/147/88/
https://www.beeradvocate.com/beer/profile/140/30420/
https://www.beeradvocate.com/beer/profile/147/84596/
https://www.beeradvocate.com/beer/profile/43153/92/
https://www.beeradvocate.com/beer/profile/26/64228/
https://www.beeradvocate.com/beer/profile/140/1904/
https://www.beeradvocate.com/beer/profile/220/74986/
http

https://www.beeradvocate.com/beer/profile/29/52480/
https://www.beeradvocate.com/beer/profile/105/81723/
https://www.beeradvocate.com/beer/profile/29/1524/
https://www.beeradvocate.com/beer/profile/306/2947/
https://www.beeradvocate.com/beer/profile/105/5747/
https://www.beeradvocate.com/beer/profile/29/653/
https://www.beeradvocate.com/beer/profile/29/3474/
https://www.beeradvocate.com/beer/profile/105/43043/
https://www.beeradvocate.com/beer/profile/306/1341/
https://www.beeradvocate.com/beer/profile/105/2768/
https://www.beeradvocate.com/beer/profile/1422/32918/
https://www.beeradvocate.com/beer/profile/1422/20705/
https://www.beeradvocate.com/beer/profile/29/1320/
https://www.beeradvocate.com/beer/profile/29/1390/
https://www.beeradvocate.com/beer/profile/105/1286/
https://www.beeradvocate.com/beer/profile/29/1642/
https://www.beeradvocate.com/beer/profile/29/37424/
https://www.beeradvocate.com/beer/profile/29/3734/
https://www.beeradvocate.com/beer/profile/306/1859/
https://www.be

...

https://www.beeradvocate.com/beer/profile/25832/96551/
https://www.beeradvocate.com/beer/profile/18899/145856/
https://www.beeradvocate.com/beer/profile/10729/62499/
https://www.beeradvocate.com/beer/profile/25832/72112/
https://www.beeradvocate.com/beer/profile/18899/54512/
https://www.beeradvocate.com/beer/profile/36150/132880/
https://www.beeradvocate.com/beer/profile/18899/80257/
https://www.beeradvocate.com/beer/profile/35990/131408/
https://www.beeradvocate.com/beer/profile/35990/133598/
https://www.beeradvocate.com/beer/profile/31635/100136/
https://www.beeradvocate.com/beer/profile/32026/105057/
https://www.beeradvocate.com/beer/profile/18899/47282/
https://www.beeradvocate.com/beer/profile/25832/87262/
https://www.beeradvocate.com/beer/profile/34185/115902/
https://www.beeradvocate.com/beer/profile/10096/28315/
https://www.beeradvocate.com/beer/profile/27940/78904/
https://www.beeradvocate.com/beer/profile/35990/133601/
https://www.beeradvocate.com/beer/profile/18899/152588/
h

https://www.beeradvocate.com/beer/profile/7548/13813/
https://www.beeradvocate.com/beer/profile/4315/22992/
https://www.beeradvocate.com/beer/profile/16031/36700/
https://www.beeradvocate.com/beer/profile/1682/4777/
https://www.beeradvocate.com/beer/profile/10245/39329/
https://www.beeradvocate.com/beer/profile/4315/22842/
https://www.beeradvocate.com/beer/profile/260/5630/
https://www.beeradvocate.com/beer/profile/260/1308/
https://www.beeradvocate.com/beer/profile/1682/5214/
https://www.beeradvocate.com/beer/profile/31792/114793/
https://www.beeradvocate.com/beer/profile/4315/109132/
https://www.beeradvocate.com/beer/profile/2520/16985/
https://www.beeradvocate.com/beer/profile/380/2360/
https://www.beeradvocate.com/beer/profile/11872/51130/
https://www.beeradvocate.com/beer/profile/2520/7469/
https://www.beeradvocate.com/beer/profile/4315/9794/
https://www.beeradvocate.com/beer/profile/9730/80050/
https://www.beeradvocate.com/beer/profile/10245/43422/
https://www.beeradvocate.com/be

https://www.beeradvocate.com/beer/profile/1675/88985/
https://www.beeradvocate.com/beer/profile/1675/58955/
https://www.beeradvocate.com/beer/profile/1675/89094/
https://www.beeradvocate.com/beer/profile/14046/53841/
https://www.beeradvocate.com/beer/profile/2467/18930/
https://www.beeradvocate.com/beer/profile/16043/69719/
https://www.beeradvocate.com/beer/profile/16043/110001/
https://www.beeradvocate.com/beer/profile/16043/117296/
https://www.beeradvocate.com/beer/profile/1675/117057/
https://www.beeradvocate.com/beer/profile/23251/71970/
https://www.beeradvocate.com/beer/profile/1675/148911/
https://www.beeradvocate.com/beer/profile/31008/91991/
https://www.beeradvocate.com/beer/profile/23251/106535/
https://www.beeradvocate.com/beer/profile/31008/92929/
https://www.beeradvocate.com/beer/profile/14046/46811/
https://www.beeradvocate.com/beer/profile/14046/53452/
https://www.beeradvocate.com/beer/profile/16043/73580/
https://www.beeradvocate.com/beer/profile/1675/4773/
https://www.b

https://www.beeradvocate.com/beer/profile/28480/222543/
https://www.beeradvocate.com/beer/profile/28480/93238/
https://www.beeradvocate.com/beer/profile/30834/89811/
https://www.beeradvocate.com/beer/profile/34646/120259/
https://www.beeradvocate.com/beer/profile/24371/68777/
https://www.beeradvocate.com/beer/profile/42312/190620/
https://www.beeradvocate.com/beer/profile/34646/120313/
https://www.beeradvocate.com/beer/profile/26974/77800/
https://www.beeradvocate.com/beer/profile/14033/42660/
https://www.beeradvocate.com/beer/profile/28480/93413/
https://www.beeradvocate.com/beer/profile/26988/74804/
https://www.beeradvocate.com/beer/profile/10541/34982/
https://www.beeradvocate.com/beer/profile/16900/72844/
https://www.beeradvocate.com/beer/profile/39943/171682/
https://www.beeradvocate.com/beer/profile/28480/78691/
https://www.beeradvocate.com/beer/profile/18412/46123/
https://www.beeradvocate.com/beer/profile/10541/73331/
https://www.beeradvocate.com/beer/profile/11174/165730/
http

https://www.beeradvocate.com/beer/profile/13307/113093/
https://www.beeradvocate.com/beer/profile/18604/153587/
https://www.beeradvocate.com/beer/profile/25314/170903/
https://www.beeradvocate.com/beer/profile/13307/98050/
https://www.beeradvocate.com/beer/profile/13307/46987/
https://www.beeradvocate.com/beer/profile/7730/106191/
https://www.beeradvocate.com/beer/profile/25314/212939/
https://www.beeradvocate.com/beer/profile/13307/58359/
https://www.beeradvocate.com/beer/profile/25314/315827/
https://www.beeradvocate.com/beer/profile/12172/221875/
https://www.beeradvocate.com/beer/profile/15711/50235/
https://www.beeradvocate.com/beer/profile/7730/279583/
https://www.beeradvocate.com/beer/profile/23680/107723/
https://www.beeradvocate.com/beer/profile/25314/282600/
https://www.beeradvocate.com/beer/profile/13307/291513/
https://www.beeradvocate.com/beer/profile/24299/83150/
https://www.beeradvocate.com/beer/profile/25314/106819/
https://www.beeradvocate.com/beer/profile/18604/213570/

In [10]:
all_beers = []
beers = list(unique_beers)

# For each beer page within start and end index pull the info and append it
def scrape_all_beers(beers, start, end):
    for beer in beers[start:end]:
        try:
            my_beer = scrape_beer_page(beer)
            all_beers.append(my_beer)
        except Exception:
            print('error with item')

            
# This utilizes multi-threading to start scraping all the beer pages from the list we got earlier
def split_processing(beers, num_splits=40):
    split_size = len(beers) // num_splits
    threads = []
    for i in range(num_splits):
        # determine the indices of the list this thread will handle
        start = i * split_size
        # special case on the last chunk to account for uneven splits
        end = None if i+1 == num_splits else (i+1) * split_size
        # create the thread
        threads.append(
            threading.Thread(target=scrape_all_beers, args=(beers, start, end))
        )
        threads[-1].start() # start the thread we just created

    # wait for all threads to finish                                            
    for t in threads:
        t.join()

split_processing(beers)

error with item
error with item
error with item
error with item
error with itemerror with item

error with item
error with item
error with item
error with itemerror with itemerror with item


error with item
error with item
error with item
error with item
error with itemerror with item

error with itemerror with item

error with item
error with item
error with item
error with itemerror with item

error with itemerror with item

error with item
error with item
error with itemerror with item

error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item
error with item


In [11]:
# This page writes the beer information into a csv file
col_names = Beer._fields
beer_data = pd.DataFrame.from_records(all_beers, columns = col_names)
beer_data.to_csv('beer_advocate.csv', index=False)